<p style = 'color:#E6E4E7 ; background-color:#00561B ; text-align:center ; font-size: 300%'> Kayak project - Get hotel data for Chateau du Haut Koenigsbourg

<p style = 'color:#00561B ; background-color:#E6E4E7 ; text-align:center ; font-size: 150%'> Project prepared by Stephanie Cotineau - #dsmpt-Paris-08

In [1]:
!pip install Scrapy -q

In [2]:
import os # For manipulate file 
import logging # For display logs
import scrapy # Import scrapy & scrapy.crawler
from scrapy.crawler import CrawlerProcess
import re

In [3]:
# Name of the file where the results will be saved
filename = "Chateau_du_Haut_Koenigsbourg_multipage.json"

# If file already exists, delete it before crawling (because Scrapy will 
# concatenate the last and new results otherwise)
if filename in os.listdir('results_hotels/'):
        os.remove('results_hotels/' + filename)

# Declare a new CrawlerProcess with some settings
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.88 Safari/537.36'
process = CrawlerProcess(settings = {
    'USER_AGENT': user_agent,
    'LOG_LEVEL': logging.ERROR,
    "FEEDS": {
        'results_hotels/' + filename : {"format": "json"},
    }
})

In [4]:
from bs4 import BeautifulSoup 
import requests

In [5]:
# Chateau_du_Haut_Kœnigsbourg - 5 first pages
offset = 0
link_offset = []
while offset != 125:
    url = f"https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB&sid=318bebf61bd36e4cdf8b55a0d0f1f93c&aid=304142&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB%26sid%3D318bebf61bd36e4cdf8b55a0d0f1f93c%26sb_price_type%3Dtotal%26%26&ss=Ch%C3%A2teau+du+Haut-K%C5%93nigsbourg%2C+Saint-Hippolyte%2C+Alsace%2C+France&is_ski_area=&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&map=1&b_h4u_keep_filters=&from_sf=1&ss_raw=chateau+du&ac_position=2&ac_langcode=fr&ac_click_type=b&dest_id=204055&dest_type=landmark&place_id_lat=48.24931&place_id_lon=7.34445&search_pageview_id=15c43bf6873c02b4&search_pageview_id=15c43bf6873c02b4&search_selected=true&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&offset={offset}"
    #print(url)
    offset = offset + 25
    link_offset.append(url)

In [6]:
link_offset

['https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB&sid=318bebf61bd36e4cdf8b55a0d0f1f93c&aid=304142&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB%26sid%3D318bebf61bd36e4cdf8b55a0d0f1f93c%26sb_price_type%3Dtotal%26%26&ss=Ch%C3%A2teau+du+Haut-K%C5%93nigsbourg%2C+Saint-Hippolyte%2C+Alsace%2C+France&is_ski_area=&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&map=1&b_h4u_keep_filters=&from_sf=1&ss_raw=chateau+du&ac_position=2&ac_langcode=fr&ac_click_type=b&dest_id=204055&dest_type=landmark&place_id_lat=48.24931&place_id_lon=7.34445&search_pageview_id=15c43bf6873c02b4&search_pageview_id=15c43bf6873c02b4&search_sel

In [7]:
class ChateauSpider(scrapy.Spider) :
    # Name of spider
    name = 'Chateau_du_Haut_Koenigsbourg'
    
    # Starting URL
    start_urls = ['https://www.booking.com/']
    
    def start_requests(self):
        for d in ['https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB&sid=318bebf61bd36e4cdf8b55a0d0f1f93c&aid=304142&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB%26sid%3D318bebf61bd36e4cdf8b55a0d0f1f93c%26sb_price_type%3Dtotal%26%26&ss=Ch%C3%A2teau+du+Haut-K%C5%93nigsbourg%2C+Saint-Hippolyte%2C+Alsace%2C+France&is_ski_area=&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&map=1&b_h4u_keep_filters=&from_sf=1&ss_raw=chateau+du&ac_position=2&ac_langcode=fr&ac_click_type=b&dest_id=204055&dest_type=landmark&place_id_lat=48.24931&place_id_lon=7.34445&search_pageview_id=15c43bf6873c02b4&search_pageview_id=15c43bf6873c02b4&search_selected=true&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&offset=0',
                  'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB&sid=318bebf61bd36e4cdf8b55a0d0f1f93c&aid=304142&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB%26sid%3D318bebf61bd36e4cdf8b55a0d0f1f93c%26sb_price_type%3Dtotal%26%26&ss=Ch%C3%A2teau+du+Haut-K%C5%93nigsbourg%2C+Saint-Hippolyte%2C+Alsace%2C+France&is_ski_area=&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&map=1&b_h4u_keep_filters=&from_sf=1&ss_raw=chateau+du&ac_position=2&ac_langcode=fr&ac_click_type=b&dest_id=204055&dest_type=landmark&place_id_lat=48.24931&place_id_lon=7.34445&search_pageview_id=15c43bf6873c02b4&search_pageview_id=15c43bf6873c02b4&search_selected=true&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&offset=25',
                  'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB&sid=318bebf61bd36e4cdf8b55a0d0f1f93c&aid=304142&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB%26sid%3D318bebf61bd36e4cdf8b55a0d0f1f93c%26sb_price_type%3Dtotal%26%26&ss=Ch%C3%A2teau+du+Haut-K%C5%93nigsbourg%2C+Saint-Hippolyte%2C+Alsace%2C+France&is_ski_area=&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&map=1&b_h4u_keep_filters=&from_sf=1&ss_raw=chateau+du&ac_position=2&ac_langcode=fr&ac_click_type=b&dest_id=204055&dest_type=landmark&place_id_lat=48.24931&place_id_lon=7.34445&search_pageview_id=15c43bf6873c02b4&search_pageview_id=15c43bf6873c02b4&search_selected=true&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&offset=50',
                  'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB&sid=318bebf61bd36e4cdf8b55a0d0f1f93c&aid=304142&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB%26sid%3D318bebf61bd36e4cdf8b55a0d0f1f93c%26sb_price_type%3Dtotal%26%26&ss=Ch%C3%A2teau+du+Haut-K%C5%93nigsbourg%2C+Saint-Hippolyte%2C+Alsace%2C+France&is_ski_area=&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&map=1&b_h4u_keep_filters=&from_sf=1&ss_raw=chateau+du&ac_position=2&ac_langcode=fr&ac_click_type=b&dest_id=204055&dest_type=landmark&place_id_lat=48.24931&place_id_lon=7.34445&search_pageview_id=15c43bf6873c02b4&search_pageview_id=15c43bf6873c02b4&search_selected=true&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&offset=75',
                  'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB&sid=318bebf61bd36e4cdf8b55a0d0f1f93c&aid=304142&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB%26sid%3D318bebf61bd36e4cdf8b55a0d0f1f93c%26sb_price_type%3Dtotal%26%26&ss=Ch%C3%A2teau+du+Haut-K%C5%93nigsbourg%2C+Saint-Hippolyte%2C+Alsace%2C+France&is_ski_area=&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&map=1&b_h4u_keep_filters=&from_sf=1&ss_raw=chateau+du&ac_position=2&ac_langcode=fr&ac_click_type=b&dest_id=204055&dest_type=landmark&place_id_lat=48.24931&place_id_lon=7.34445&search_pageview_id=15c43bf6873c02b4&search_pageview_id=15c43bf6873c02b4&search_selected=true&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&offset=100']:
            yield scrapy.Request(d,callback=self.search)
    
    # Callback used after search launched
    def search(self, response):
        hotels = response.css('div.a826ba81c4.fe821aea6c.fa2f36ad22.afd256fc79.d08f526e0d.ed11e24d01.da89aeb942')
        for hotel in hotels:
            yield {
                'location' : hotel.css('span.f4bd0794db.b4273d69aa::text').get(),
                'hotel_url' : hotel.css('a.e13098a59f').attrib['href'],
                'hotel_name' : hotel.css('div.fcab3ed991.a23c043802::text').get(),
                'hotel_rating' : hotel.css('div.b5cd09854e.d10a6220b4::text').get(),
                'hotel_description_short' : hotel.css('div.d8eab2cf7f::text').get()
            }

In [8]:
# Start the crawling using the spider defined above
process.crawl(ChateauSpider)
process.start()

In [9]:
import pandas as pd

In [10]:
chateau = pd.read_json('results_hotels/Chateau_du_Haut_Koenigsbourg_multipage.json')

In [11]:
len(chateau)

125

In [12]:
chateau.head(55)

,location,hotel_url,hotel_name,hotel_rating,hotel_description_short
0,Ammerschwihr,https://www.booking.com/hotel/fr/alsacebnb.fr....,Alsacebnb - Gîte 12 personnes dans le vignoble...,"9,6",Doté d'une connexion Wi-Fi gratuite dans l'ens...
1,Ostheim,https://www.booking.com/hotel/fr/gite-d-alsace...,Gîte d’Alsace l’Escale,"9,4",Hébergement géré par un particulier
2,Breitenau,https://www.booking.com/hotel/fr/a-l-39-ombre-...,A l'ombre du verger,"9,5",Hébergement géré par un particulier
3,Riquewihr,https://www.booking.com/hotel/fr/le-schoenenbo...,Best Western Hotel & SPA Le Schoenenbourg,"8,6","Entouré de vignobles, le Best Western Hotel Le..."
4,Sainte-Marie-aux-Mines,https://www.booking.com/hotel/fr/la-pouponiere...,La Pouponiere,"9,0",Hébergement géré par un particulier
5,Bernardvillé,https://www.booking.com/hotel/fr/les-jardins-d...,Maison d'Hôtes Les Jardins de Madeleine,"9,9","Située à Bernardvillé, à 34 km de Riquewihr, l..."
6,Riquewihr,https://www.booking.com/hotel/fr/le-riquewihr....,Hôtel Le Riquewihr,"8,1","Situé dans la ville médiévale de Riquewihr, l'..."
7,Riquewihr,https://www.booking.com/hotel/fr/a-l-oriel.fr....,Hotel à l'Oriel,"8,4","L'Oriel est situé en plein cœur de Riquewihr, ..."
8,Triembach-au-Val,https://www.booking.com/hotel/fr/chambre-d-39-...,"Chambre d'Hôtes ""Au Vigneron""","9,4",Hébergement géré par un particulier
9,Ribeauvillé,https://www.booking.com/hotel/fr/restaurant-au...,Hotel SPA Restaurant Au Cheval Blanc,"8,1",L'Hotel SPA Restaurant Au Cheval Blanc est sit...


In [13]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.0) AppleWebKit/535.1 (KHTML, like Gecko) Chrome/13.0.782.112 Safari/535.1'}
result_ll =[]
for link in chateau['hotel_url']:
    page = requests.get(link, headers = headers)
    soup_link = BeautifulSoup(page.content, "lxml")
    lat_lon = soup_link.find_all(id = "hotel_address")
    for ll in lat_lon : 
        #print(ll['data-atlas-latlng'])
        result_ll.append(ll['data-atlas-latlng'])

In [14]:
len(result_ll)

125

In [15]:
chateau['Latitude_Longitude'] = result_ll

In [16]:
lat = []
lon = []

# For each row in a varible,
for row in chateau['Latitude_Longitude']:
    # Try to,
    try:
        # Split the row by comma and append
        # everything before the comma to lat
        lat.append(row.split(',')[0])
        # Split the row by comma and append
        # everything after the comma to lon
        lon.append(row.split(',')[1])
    # But if you get an error
    except:
        # append a missing value to lat
        lat.append(np.NaN)
        # append a missing value to lon
        lon.append(np.NaN)

# Create two new columns from lat and lon
chateau['latitude'] = lat
chateau['longitude'] = lon

In [17]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.0) AppleWebKit/535.1 (KHTML, like Gecko) Chrome/13.0.782.112 Safari/535.1'}
result_long_description =[]
for link in chateau['hotel_url']:
    page = requests.get(link, headers = headers)
    soup_link = BeautifulSoup(page.content, "lxml")
    long_description = soup_link.find_all(id = "property_description_content") 
    for ld in long_description : 
        result_long_description.append(ld.text)

In [18]:
len(result_long_description)

125

In [19]:
chateau['hotel_description_long'] = result_long_description

In [20]:
chateau.head(55)

,location,hotel_url,hotel_name,hotel_rating,hotel_description_short,Latitude_Longitude,latitude,longitude,hotel_description_long
0,Ammerschwihr,https://www.booking.com/hotel/fr/alsacebnb.fr....,Alsacebnb - Gîte 12 personnes dans le vignoble...,"9,6",Doté d'une connexion Wi-Fi gratuite dans l'ens...,"48.12672300,7.28661300",48.12672300,7.28661300,\nDoté d'une connexion Wi-Fi gratuite dans l'e...
1,Ostheim,https://www.booking.com/hotel/fr/gite-d-alsace...,Gîte d’Alsace l’Escale,"9,4",Hébergement géré par un particulier,"48.15967462,7.36115874",48.15967462,7.36115874,\nVous pouvez bénéficier d'une réduction Geniu...
2,Breitenau,https://www.booking.com/hotel/fr/a-l-39-ombre-...,A l'ombre du verger,"9,5",Hébergement géré par un particulier,"48.32252300,7.28986700",48.32252300,7.28986700,"\nSitué à Breitenau, l'établissement A l'ombre..."
3,Riquewihr,https://www.booking.com/hotel/fr/le-schoenenbo...,Best Western Hotel & SPA Le Schoenenbourg,"8,6","Entouré de vignobles, le Best Western Hotel Le...","48.16697453,7.30060011",48.16697453,7.30060011,"\nEntouré de vignobles, le Best Western Hotel ..."
4,Sainte-Marie-aux-Mines,https://www.booking.com/hotel/fr/la-pouponiere...,La Pouponiere,"9,0",Hébergement géré par un particulier,"48.24620200,7.20012100",48.24620200,7.20012100,\nDotée d'une connexion Wi-Fi gratuite et offr...
5,Bernardvillé,https://www.booking.com/hotel/fr/les-jardins-d...,Maison d'Hôtes Les Jardins de Madeleine,"9,9","Située à Bernardvillé, à 34 km de Riquewihr, l...","48.36873540,7.42584874",48.36873540,7.42584874,\nVous pouvez bénéficier d'une réduction Geniu...
6,Riquewihr,https://www.booking.com/hotel/fr/le-riquewihr....,Hôtel Le Riquewihr,"8,1","Situé dans la ville médiévale de Riquewihr, l'...","48.16602996,7.30618447",48.16602996,7.30618447,\nVous pouvez bénéficier d'une réduction Geniu...
7,Riquewihr,https://www.booking.com/hotel/fr/a-l-oriel.fr....,Hotel à l'Oriel,"8,4","L'Oriel est situé en plein cœur de Riquewihr, ...","48.16663999,7.29892910",48.16663999,7.29892910,\nVous pouvez bénéficier d'une réduction Geniu...
8,Triembach-au-Val,https://www.booking.com/hotel/fr/chambre-d-39-...,"Chambre d'Hôtes ""Au Vigneron""","9,4",Hébergement géré par un particulier,"48.33785787,7.32529467",48.33785787,7.32529467,\nVous pouvez bénéficier d'une réduction Geniu...
9,Ribeauvillé,https://www.booking.com/hotel/fr/restaurant-au...,Hotel SPA Restaurant Au Cheval Blanc,"8,1",L'Hotel SPA Restaurant Au Cheval Blanc est sit...,"48.19661034,7.31340766",48.19661034,7.31340766,\nVous pouvez bénéficier d'une réduction Geniu...


In [21]:
chateau['hotel_description_long'] = chateau['hotel_description_long'].replace('\n','', regex=True)

In [22]:
chateau.head(5)

,location,hotel_url,hotel_name,hotel_rating,hotel_description_short,Latitude_Longitude,latitude,longitude,hotel_description_long
0,Ammerschwihr,https://www.booking.com/hotel/fr/alsacebnb.fr....,Alsacebnb - Gîte 12 personnes dans le vignoble...,"9,6",Doté d'une connexion Wi-Fi gratuite dans l'ens...,"48.12672300,7.28661300",48.12672300,7.28661300,Doté d'une connexion Wi-Fi gratuite dans l'ens...
1,Ostheim,https://www.booking.com/hotel/fr/gite-d-alsace...,Gîte d’Alsace l’Escale,"9,4",Hébergement géré par un particulier,"48.15967462,7.36115874",48.15967462,7.36115874,Vous pouvez bénéficier d'une réduction Genius ...
2,Breitenau,https://www.booking.com/hotel/fr/a-l-39-ombre-...,A l'ombre du verger,"9,5",Hébergement géré par un particulier,"48.32252300,7.28986700",48.32252300,7.28986700,"Situé à Breitenau, l'établissement A l'ombre d..."
3,Riquewihr,https://www.booking.com/hotel/fr/le-schoenenbo...,Best Western Hotel & SPA Le Schoenenbourg,"8,6","Entouré de vignobles, le Best Western Hotel Le...","48.16697453,7.30060011",48.16697453,7.30060011,"Entouré de vignobles, le Best Western Hotel Le..."
4,Sainte-Marie-aux-Mines,https://www.booking.com/hotel/fr/la-pouponiere...,La Pouponiere,"9,0",Hébergement géré par un particulier,"48.24620200,7.20012100",48.24620200,7.20012100,Dotée d'une connexion Wi-Fi gratuite et offran...


In [23]:
chateau['hotel_description_long'][55]

"Situé à Sélestat, le Carpe Diem Guest House dispose d'une terrasse. Il propose un restaurant, un service d'étage, un salon commun et une connexion Wi-Fi gratuite dans l'ensemble de ses locaux. Cette maison d'hôtes propose des chambres familiales.Toutes les chambres comprennent un bureau, une télévision à écran plat, une salle de bains privative, du linge de lit et des serviettes. Les chambres sont équipées d'une bouilloire. Certaines comprennent une kitchenette avec un micro-ondes, un four et des plaques de cuisson. Tous les logements du Carpe Diem Guest House comprennent la climatisation et une armoire.Un petit-déjeuner continental est servi sur place.Vous pourrez vous détendre dans le spa et centre de bien-être, comprenant un hammam et un bain à remous, ou dans le jardin équipé d'une aire de jeux pour enfants et d'un barbecue. Vous pourrez jouer au billard et au ping-pong sur place. La région est prisée des amateurs de randonnée et de vélo.Vous séjournerez à 38 km de Rust et à 23 km

In [24]:
# Clean location column
chateau['location'] = ["Chateau du Haut Koenigsbourg" if value!="Chateau du Haut Koenigsbourg" else value for value in chateau["location"]]

In [25]:
chateau.head()

,location,hotel_url,hotel_name,hotel_rating,hotel_description_short,Latitude_Longitude,latitude,longitude,hotel_description_long
0,Chateau du Haut Koenigsbourg,https://www.booking.com/hotel/fr/alsacebnb.fr....,Alsacebnb - Gîte 12 personnes dans le vignoble...,"9,6",Doté d'une connexion Wi-Fi gratuite dans l'ens...,"48.12672300,7.28661300",48.12672300,7.28661300,Doté d'une connexion Wi-Fi gratuite dans l'ens...
1,Chateau du Haut Koenigsbourg,https://www.booking.com/hotel/fr/gite-d-alsace...,Gîte d’Alsace l’Escale,"9,4",Hébergement géré par un particulier,"48.15967462,7.36115874",48.15967462,7.36115874,Vous pouvez bénéficier d'une réduction Genius ...
2,Chateau du Haut Koenigsbourg,https://www.booking.com/hotel/fr/a-l-39-ombre-...,A l'ombre du verger,"9,5",Hébergement géré par un particulier,"48.32252300,7.28986700",48.32252300,7.28986700,"Situé à Breitenau, l'établissement A l'ombre d..."
3,Chateau du Haut Koenigsbourg,https://www.booking.com/hotel/fr/le-schoenenbo...,Best Western Hotel & SPA Le Schoenenbourg,"8,6","Entouré de vignobles, le Best Western Hotel Le...","48.16697453,7.30060011",48.16697453,7.30060011,"Entouré de vignobles, le Best Western Hotel Le..."
4,Chateau du Haut Koenigsbourg,https://www.booking.com/hotel/fr/la-pouponiere...,La Pouponiere,"9,0",Hébergement géré par un particulier,"48.24620200,7.20012100",48.24620200,7.20012100,Dotée d'une connexion Wi-Fi gratuite et offran...


In [26]:
chateau.to_csv("Chateau_du_Haut_Koenigsbourg.csv")